# Overview

Interfereometer data (visibilities) and single dish data (spectrum).

MSv2 and CASA documentation used in creating the schema:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf
- Ephemeris Data in CASA: https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data
- VLBI MSv2 extension: https://casacore.github.io/casacore-notes/265.pdf

We reused descriptions from the MSv2 (add thank you).


Use cases

- 
- 
-
-
- No pulsars


Tests


Offline processing (MS)


What is missing:
Interferometer Model
Beam models (attched to )


How data is partitioned: 

The data must be partitioned at least on Observation, Spectral Windows and Polarization setup (data can be represented as a n dimensional array).

Optional partining [FIELD_ID, ...]


The default is [OBSERVATION, SPECTRAL WINDOW, POLARIZATION setup , FIELD]

For mosaics where the phase center is rapidly varying such as VLASS do not partion on field.



Add a diagram of on dish and in memory data.


<!--Link to google drawing: https://docs.google.com/drawings/d/1afPe5oro26NMTkAKpK9iif0adNA0B4R9otLookOixvI/edit?usp=sharing -->

Will eventually replace Python-casacore.
Asdm library does not exist yet.

https://docs.google.com/drawings/d/1P9AI3D4VzPGw8O72dPBkz1iED8-ITM96anLeru0dvgM/edit?usp=sharing

<img src="https://docs.google.com/drawings/d/e/2PACX-1vQVgjF5xNeIv8gpi2G3R8JXw2bNkVIUXdizIZluCGdnHc4z79ryW2fNUycJAd_CQh9sXLwdlx1oiAAX/pub?w=690&amp;h=510">


Types of correlated datasets visibility (single dish) and spectrum (single dish)

## Processing Set Data Structure

## Measurement Set v4

### Measurement Set v2 Deprecated 

### Data Groups (optional)

Data reuse.